# sphinx visualization demo

本demo用于直观调试数据预处理流程和模型inference流程

进行模型训练和生成测试结果请见launcher.py

#### 使用说明

1. 先执行以下2个cell和初始化部分的3个cell
2. 后续三个测试部分可以选择运行，不必按顺序执行
3. 每个测试部分可以多次执行

In [ ]:
import os
os.chdir('..')

import cv2
import numpy as np
import tensorflow as tf

import utils as ut
from config import Config
from preprocess.datagen import DataGenerator, _relative_points, _pad_img
from models.sphinx import SphinxModel
from scripts.test import Tester
from scripts.train import Trainer

from matplotlib import pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
def predict_one(model, images, general_category, new_p, weight, orig_size, image_visual):
    """
    params
         model: inference model
         images: padded and resized image, shape [1, img_size, img_size]
         general_category: 'top' or 'bottom'
         new_p: padded ground truth, shape [num_points, 2]
         weight: point weight, shape [num_points]
         orig_size: raw image size
         image_visual: padded raw image, shape [1, orig_size, orig_size]
    """
    # get prediction
    pred = model.Session.run(model.output[2], feed_dict={model.img: images})
    pred = pred[0]

    # calculate predictions from heatmap
    pred_idxes = np.zeros((model.num_points, 2), dtype=np.float32)
    total_dist = 0.0
    for j in range(model.num_points):
        if weight[j] == 1:
            index = np.unravel_index(pred[:, :, j].argmax(), (model.out_size, model.out_size))
            index = (index[1], index[0])
            pred_idx = np.array(index) / model.out_size * orig_size
            pred_idxes[j, :] = pred_idx
            total_dist += np.linalg.norm(pred_idx - new_p[j])

    # visualize prediction
    pred_hm_visual = dataset._generate_hm(orig_size, model.out_size, pred_idxes, weight, keep_invisible=False, sigma=12)
    pred_hm_visual = np.sum(pred_hm_visual, 2) * 255
    pred_hm_visual = cv2.resize(pred_hm_visual, (orig_size, orig_size), interpolation=cv2.INTER_LINEAR)
    pred_hm_visual = np.expand_dims(pred_hm_visual, 2)
    image_visual = image_visual + pred_hm_visual.astype(np.int32)  
    plt.imshow(image_visual)
    plt.show()

    # calc normalized average error
    if general_category == 'top':
        norm_idx1 = new_p[5]
        norm_idx2 = new_p[6]
    else:
        norm_idx1 = new_p[0]
        norm_idx2 = new_p[1]

    norm_dist = np.linalg.norm(norm_idx2 - norm_idx1)
    error = total_dist / norm_dist / np.sum(np.count_nonzero(weight))
    print("image error:", error)

## 初始化数据集和模型

#### 注意修改以下参数：
1. CUDA_VISIBLE_DEVICES
2. config配置文件

#### 关于初始化和硬件占用
1. 模型初始化后，关闭该页面不会终止显存占用，再次打开页面也不需要重新初始化
2. 如果要释放显存，需要在页面内restart kernel，或在jupyter首页shutdown该页面

In [ ]:
cfg = Config()
cfg.total_points = len(cfg.points_list)
cfg.load = "trousers_1"

dataset = DataGenerator(cfg)
dataset.generate_set(train = True)
# dataset.generate_set(train = False)

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
model = SphinxModel(cfg)
model.img = tf.placeholder(tf.float32, (None, cfg.img_size, cfg.img_size, 3))
model.out_size = cfg.hm_size * cfg.out_rate
model.output = model.graph(model.img, False)

In [ ]:
with tf.name_scope('Session'):
    model.Session = tf.Session()
    
    assert (cfg.load is not None), 'Config "Load" must be specified.'
    saver = tf.train.Saver(max_to_keep=10)
    saver.restore(model.Session, os.path.join(cfg.saver_dir, cfg.load))

## 在训练集上进行测试

#### 主要是检查data augmentation的效果
#### 每次测试1张图片

In [ ]:
train_gen = dataset.generator(cfg.img_size, cfg.hm_size, 1, cfg.nStacks, 'train')

In [ ]:
# generate augmented data
img, hm0, hm1, hm2, weight = next(train_gen)
image = img[0]
hm = hm2[0]
weight = weight[0] 

raw_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
raw_image = cv2.resize(raw_image, (model.out_size, model.out_size), interpolation=cv2.INTER_LINEAR)

# visualize ground truth
gt_hm_visual = np.sum(hm, 2) * 255
gt_hm_visual = np.expand_dims(gt_hm_visual, 2)
gt_visual = raw_image + gt_hm_visual.astype(np.int32)
plt.imshow(gt_visual)
plt.show()

# calculate ground truth from gt_maps
new_p = np.zeros((model.num_points, 2))
for i in range(model.num_points):
    if weight[i] == 1:
        this_map = hm[:, :, i]
        pred_idx = np.reshape(np.array(np.array(np.where(this_map == np.max(this_map)))), [2])
        pred_idx[0], pred_idx[1] = pred_idx[1], pred_idx[0]
        new_p[i] = pred_idx

# visualize prediction
predict_one(model, img, 'bottom', new_p, weight, model.out_size, raw_image)

## 在验证集上进行测试

#### 每次测试1张图片。修改i控制图片位置
#### 为方便调试，这里不使用generator

In [ ]:
i = 0

In [ ]:
# generate augmented data
img, hm0, hm1, hm2, weight = next(train_gen)
image = img[0]
hm = hm2[0]
weight = weight[0] 

raw_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
raw_image = cv2.resize(raw_image, (model.out_size, model.out_size), interpolation=cv2.INTER_LINEAR)

# visualize ground truth
gt_hm_visual = np.sum(hm, 2) * 255
gt_hm_visual = np.expand_dims(gt_hm_visual, 2)
gt_visual = raw_image + gt_hm_visual.astype(np.int32)
plt.imshow(gt_visual)
plt.show()

# calculate ground truth from gt_maps
new_p = np.zeros((model.num_points, 2))
for i in range(model.num_points):
    if weight[i] == 1:
        this_map = hm[:, :, i]
        pred_idx = np.reshape(np.array(np.array(np.where(this_map == np.max(this_map)))), [2])
        pred_idx[0], pred_idx[1] = pred_idx[1], pred_idx[0]
        new_p[i] = pred_idx

# visualize prediction
predict_one(model, img, 'bottom', new_p, weight, model.out_size, raw_image)

## 在测试集上进行测试

#### 重新初始化了dataset. 使用test_generator进行测试
#### 每次测试1张图片

In [ ]:
test_gen = dataset.test_generator(cfg.img_size, 1)

In [ ]:
# Traversal the test set
images, categories, offsets, names, sizes = next(test_gen)
prediction = model.Session.run(model.output[3], feed_dict={model.img: images})
hms = prediction

# init
pred_idxes = np.zeros((cfg.total_points, 2), dtype=np.float32)
img = images[0]
hm = hms[0]
offset = offsets[0]
category = categories[0]
name = names[0]
size = sizes[0]

# predict
write_line = [name, category]
cnt = 0
for j in range(cfg.total_points):
    if ut.VALID_POINTS[cfg.is_top][j]:
        if ut.VALID_POSITION[category][j] == 1:
            index = np.unravel_index(hm[:, :, cnt].argmax(), (model.out_size, model.out_size))
            index = (index[1], index[0])
            point = np.array(index) / model.out_size * size
            pred_idxes[j] = point
            point -= offset
            write_line.append(str(int(round(point[0]))) + '_' + str(int(round(point[1]))) + '_1')
        else:
            write_line.append('-1_-1_-1')
        cnt += 1
    else:
        write_line.append('-1_-1_-1')
print(write_line)

print(pred_idxes)

# visualize prediction
pred_hm_visual = np.zeros((size, size, 3))
pred_hm = dataset._generate_hm(size, size, pred_idxes, ut.VALID_POSITION[category], keep_invisible=False, sigma=12)
pred_hm = np.sum(pred_hm, 2) * 255
pred_hm_visual[:, :, 1] = pred_hm
pred_visual = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (size, size), interpolation=cv2.INTER_LINEAR)
pred_visual = pred_visual + pred_hm_visual.astype(np.int32)
# show results
plt.imshow(pred_visual)
plt.show()